# May Mobility (Data Scientist)

## Rough Idea of how the route looks

I don't believe that this is the actual route.

As I comb through more of the data, I'll get a better understanding of what order the stops go.

I just ploted the lat/lon in order from the appendix to give myself a visual aid.

![rough_image](./resources/pics/rough_idea_route.png)


### Point of Interest (PoI)

| Stop       | Description                      | Latitude | Longitude |
|:-----------|:--------------------------------:|----------|-----------|
| Bus        | Bus stop on a major transit line | 39.77285 | -86.16168 |
| Dentist    | School of Dentistry              | 39.77467 | -86.17895 |
| Doctor     | Pediatrician’s office            | 39.77926 | -86.17496 |
| Admin      | Administrative building          | 39.77459 | -86.17433 |
| Hospital   | Campus hospital                  | 39.77567 | -86.17557 |
| Lime       | Bus stop on campus               | 39.77473 | -86.18376 |
| Parking    | Campus parking lot               | 39.77882 | -86.18121 |
| School     | School of Art and Design         | 39.77148 | -86.17148 |
| University | University lecture hall          | 39.77271 | -86.17575 |


## Read Data in

In [24]:
import pandas as pd
import numpy as np
# QoL for viewing df output
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# read-in *.csv files
pickups_df   = pd.read_csv("resources/csv/Data_Science_pickups.csv", na_values= np.nan)
ridership_df = pd.read_csv("resources/csv/Data_Science_site_ridership.csv", na_values= np.nan)

# display head of dataframes
print(pickups_df.head())
print(ridership_df.head())

   row_id            timestamp  pickup  dropoff    stop vehicle      time        date name
0       1  2021-11-01 07:10:54       1        0     Bus  Marble  07:00:00  2021-11-01   ES
1       2  2021-11-01 07:51:13       1        0     Bus  Marble  07:50:00  2021-11-01   ES
2       3  2021-11-01 08:02:13       1        0    Lime  Marble  08:01:00  2021-11-01   ES
3       4  2021-11-01 08:41:16       1        0  Doctor   Motto  08:41:00  2021-11-01   CM
4       5  2021-11-01 09:24:10       1        0     Bus    Myao  09:22:00  2021-11-01   CM
             timestamp  pickup  dropoff    stop vehicle      time        date name
0  2021-06-03 13:08:10     1.0      0.0     Bus   Motto  13:05:00  2021-06-03   JR
1  2021-06-03 13:31:41     0.0      1.0     Bus   Motto  13:31:43  2021-06-03   JR
2  2021-06-04 11:06:02     1.0      0.0  School   Motto  11:03:00  2021-06-04   CM
3  2021-06-04 11:07:48     0.0      1.0     Bus   Motto  11:07:00  2021-06-04   CM
4  2021-06-04 12:43:54     1.0      0.0

## Inspect and convert data types

In [25]:
print('pickups data frame\n')
pickups_df.info()

pickups data frame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363 entries, 0 to 362
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   row_id     363 non-null    int64 
 1   timestamp  363 non-null    object
 2   pickup     363 non-null    int64 
 3   dropoff    363 non-null    int64 
 4   stop       363 non-null    object
 5   vehicle    363 non-null    object
 6   time       363 non-null    object
 7   date       363 non-null    object
 8   name       363 non-null    object
dtypes: int64(3), object(6)
memory usage: 25.6+ KB


In [26]:
# Data type changes in pickups_df
# timestamp    object
pickups_df["timestamp"] = pd.to_datetime(pickups_df["timestamp"], format="%Y-%m-%d %H:%M:%S")

# stop         object
pickups_df["stop"] = pickups_df["stop"].astype("category")

# vehicle      object
pickups_df["vehicle"] = pickups_df["vehicle"].astype("category")

# date         object
pickups_df["date"] = pd.to_datetime(pickups_df["date"], format="%Y-%m-%d")

# name         object
pickups_df["name"] = pickups_df["name"].astype("category")

pickups_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363 entries, 0 to 362
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   row_id     363 non-null    int64         
 1   timestamp  363 non-null    datetime64[ns]
 2   pickup     363 non-null    int64         
 3   dropoff    363 non-null    int64         
 4   stop       363 non-null    category      
 5   vehicle    363 non-null    category      
 6   time       363 non-null    object        
 7   date       363 non-null    datetime64[ns]
 8   name       363 non-null    category      
dtypes: category(3), datetime64[ns](2), int64(3), object(1)
memory usage: 19.1+ KB


In [27]:
print('ridership data frame\n')
ridership_df.info()

ridership data frame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4352 entries, 0 to 4351
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  4352 non-null   object 
 1   pickup     4340 non-null   float64
 2   dropoff    4337 non-null   float64
 3   stop       4352 non-null   object 
 4   vehicle    4352 non-null   object 
 5   time       4352 non-null   object 
 6   date       4352 non-null   object 
 7   name       4352 non-null   object 
dtypes: float64(2), object(6)
memory usage: 272.1+ KB


In [28]:
# Data type changes in ridership_df
# timestamp     object
ridership_df["timestamp"] = pd.to_datetime(ridership_df["timestamp"], format="%Y-%m-%d %H:%M:%S")

# # pickup       float64 
# Should be int64
# ridership_df["pickup"] = ridership_df["pickup"]

# # dropoff      float64
# Should be int64
# ridership_df["dropoff"] = ridership_df["dropoff"]

# stop          object
ridership_df["stop"] = ridership_df["stop"].astype("category")

# vehicle       object
ridership_df["vehicle"] = ridership_df["vehicle"].astype("category")

# date          object
ridership_df["date"] = pd.to_datetime(ridership_df["date"], format="%Y-%m-%d")

# name          object
ridership_df["name"] = ridership_df["name"].astype("category")

ridership_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4352 entries, 0 to 4351
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  4352 non-null   datetime64[ns]
 1   pickup     4340 non-null   float64       
 2   dropoff    4337 non-null   float64       
 3   stop       4352 non-null   category      
 4   vehicle    4352 non-null   category      
 5   time       4352 non-null   object        
 6   date       4352 non-null   datetime64[ns]
 7   name       4352 non-null   category      
dtypes: category(3), datetime64[ns](2), float64(2), object(1)
memory usage: 184.8+ KB


In [29]:
# Find out why its null
#    RangeIndex: 4352 entries, 0 to 4351
# 1   pickup     4340 non-null   float64
# 2   dropoff    4337 non-null   float64

print(f'Missing pickup data {ridership_df["pickup"].isnull().sum()} out of {ridership_df.shape[0]} records, {ridership_df["pickup"].isnull().sum()/ridership_df.shape[0]:.2%} missing')
print(f'Missing dropoff data {ridership_df["dropoff"].isnull().sum()} out of {ridership_df.shape[0]} records, {ridership_df["dropoff"].isnull().sum()/ridership_df.shape[0]:.2%} missing')

Missing pickup data 12 out of 4352 records, 0.28% missing
Missing dropoff data 15 out of 4352 records, 0.34% missing


In [30]:
# inspect missing data from pickups
null_ridership_pickup = ridership_df[ridership_df["pickup"].isnull()]
null_ridership_pickup

,timestamp,pickup,dropoff,stop,vehicle,time,date,name
79,2021-06-17 14:09:37,NaN,1.0,Hospital,Mette,14:06:00,2021-06-17,MV
81,2021-06-17 16:52:03,NaN,1.0,Doctor,Motto,16:51:00,2021-06-17,JR
83,2021-06-17 20:25:00,NaN,1.0,Bus,Mette,17:03:00,2021-06-17,JR
85,2021-06-18 11:23:45,NaN,1.0,Parking,Myao,11:16:00,2021-06-18,CM
87,2021-06-18 14:16:28,NaN,1.0,School,Mette,14:16:00,2021-06-18,MN
89,2021-06-18 17:08:13,NaN,1.0,University,Motto,17:07:00,2021-06-18,JR
91,2021-06-21 11:50:34,NaN,1.0,University,Myao,11:49:00,2021-06-21,CM
93,2021-06-21 11:51:33,NaN,1.0,Bus,Mette,09:15:00,2021-06-21,CM
95,2021-06-21 11:52:30,NaN,1.0,Bus,Mette,09:39:00,2021-06-21,CM
97,2021-06-21 11:53:31,NaN,1.0,Doctor,Myao,10:33:00,2021-06-21,CM


In [31]:
# inspect missing data from pickups
null_ridership_dropoff = ridership_df[ridership_df["dropoff"].isnull()]
null_ridership_dropoff

,timestamp,pickup,dropoff,stop,vehicle,time,date,name
78,2021-06-17 14:02:53,1.0,NaN,Lime,Mette,14:00:00,2021-06-17,MV
80,2021-06-17 16:49:04,1.0,NaN,Bus,Motto,16:40:00,2021-06-17,JR
82,2021-06-17 16:53:10,1.0,NaN,Doctor,Mette,16:52:00,2021-06-17,JR
84,2021-06-18 11:05:49,1.0,NaN,Bus,Myao,11:05:00,2021-06-18,CM
86,2021-06-18 14:12:59,1.0,NaN,Lime,Mette,14:10:00,2021-06-18,MN
88,2021-06-18 17:05:47,1.0,NaN,Doctor,Motto,17:04:00,2021-06-18,JR
90,2021-06-21 11:26:12,1.0,NaN,University,Myao,11:25:00,2021-06-21,CM
92,2021-06-21 11:51:08,1.0,NaN,Lime,Mette,08:56:00,2021-06-21,CM
94,2021-06-21 11:52:02,1.0,NaN,Doctor,Mette,09:29:00,2021-06-21,CM
96,2021-06-21 11:53:04,1.0,NaN,Bus,Myao,10:21:00,2021-06-21,CM


In [146]:
# investage errors/nulls/missing data.
record_counts_per = ridership_df["name"].value_counts()
total_errors_per =  ridership_df[["pickup","dropoff"]].isnull().groupby(ridership_df["name"]).sum().sum(axis=1)
percent_error_per = total_errors_per/record_counts_per

print(f"Total errors or miussing data in ridership_df is {total_errors_per.sum()}.")

errors_df = pd.DataFrame({"percent_error":percent_error_per,
                          "errors":total_errors_per,
                          "records":record_counts_per})

errors_df.loc[:, "percent_error"] = errors_df["percent_error"].map('{:.2%}'.format)
errors_df[errors_df["errors"] != 0].sort_values("records",ascending=False)

Total errors or miussing data in ridership_df is 27.


,percent_error,errors,records
JR,0.39%,6,1557
CM,1.07%,16,1497
MV,4.17%,2,48
MN,25.00%,2,8
HK,25.00%,1,4


In [183]:
# check to see if there is any simultaneous pickups and dropoffs
# subset removing NaN's
NaN_idx = null_ridership_dropoff.index.union(null_ridership_pickup.index)
NaN_df = ridership_df.drop(NaN_idx)

# sanity check!
print(NaN_df.shape[0])
print(ridership_df.shape[0])


4325
4352


### Convert Vars to proper dtypes

<table>
<tr><th>Pickup Data Types </th><th>Ridership Data Types</th></tr>
<tr><td>

| Var         |       Before  |       After     |
|:------------|:--------------|:----------------|
|row_id       | int64         | int64           |
|timestamp    | object        | datetime64[ns]  |   
|pickup       | int64         | int64           |
|dropoff      | int64         | int64           |
|stop         | object        | category        |
|vehicle      | object        | category        |
|time         | object        | object          |
|date         | object        | datetime64[ns]  |   
|name         | object        | category        |

</td><td>

| Var         |       Before  |       After    |
|:------------|:--------------|:---------------|
|timestamp    | object        | datetime64[ns] |
|pickup       | float64       | float64        |
|dropoff      | float64       | float64        |
|stop         | object        | category       |
|vehicle      | object        | category       |
|time         | object        | object         |
|date         | object        | datetime64[ns] |
|name         | object        | category       |

</td></tr> </table>